# Greek 120 Visualizations

## Configuring Libraries for the Almond Kernel

First, we'll make a bintray repository with libraries available to your almond kernel.

In [23]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")

interp.repositories() ++= Seq(myBT)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [24]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import $ivy.`com.github.tototoshi::scala-csv:1.3.6`
import com.github.tototoshi.csv._

// if you want to have the plots available without an internet connection:
init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.`edu.holycross.shot::ohco2:10.18.2`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:6.0.4`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`
import $ivy.`edu.holycross.shot::greek:2.3.3`

## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [25]:
import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source

import java.text.SimpleDateFormat
import java.util.Date

// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.greek._




## Useful Functions

Save a string to a named file:

In [26]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

Like `.split`, but preserving the character we split on:

In [27]:
def splitWithSplitter(text: String, puncs: String): Vector[String] = {
	//val regexWithSplitter = s"((?<=${puncs})|(?=${puncs}))"
    val regexWithSplitter = s"((?<=${puncs}))"
	text.split(regexWithSplitter).toVector.filter(_.size > 0)
}

Pretty Print Things:

In [28]:
def showMe(v:Any):Unit = {
  v match {
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

## Classes for this Data

There is a file for each day. Each file consists of rows for reporting regions. These may be provinces, countries, or regions. Each row has data-cells. 

We will build classes from the inside-out. We are ignoring provinces for now.

In [29]:
case class DataRow( editor: String, allenText: String, ctsUrn: CtsUrn, surfUrn: Cite2Urn, imageRoi: Option[Cite2Urn])

case class IndexedDataRow( index: Int, data: DataRow)

## Load Some Data

Load current data from our shared Google Spreadsheet.

The data is in `.csv`, and the fields included change over time, so we have to do this the hard way, with a real CSV library.

In [30]:

val dataUrl: String = "https://docs.google.com/spreadsheets/d/1zDzKz99n3TYS0Uzy79tpfX6TArIyXPZ9JQI0zPq8NQU/export?format=csv"


def getNParse(url: String ): List[Map[String, String]] = {
    val reader = CSVReader.open(scala.io.Source.fromURL(url))
    reader.allWithHeaders()
}


def csvMapToVecDataRow( csv: List[Map[String, String]]): Vector[DataRow] = {
     csv.map( dm => {
        val editor: String = dm("Editor")
        val allenText: String = dm("allenText")
        val ctsUrn: CtsUrn = CtsUrn(dm("ctsUrn"))
        val surfUrn: Cite2Urn = {
             Cite2Urn(s"""urn:cite2:hmt:msA.v1:${dm("surfUrn")}""")
         }
        val imageRoi: Option[Cite2Urn] = {
            if ( dm("imageRoi") == "None" ) None
            else Some(Cite2Urn(dm("imageRoi")))
        }
        
        DataRow(editor, allenText, ctsUrn, surfUrn, imageRoi)
    }).toVector
}


val csv: List[Map[String, String]] = getNParse(dataUrl)
val hmtData: Vector[IndexedDataRow] = csvMapToVecDataRow(csv).zipWithIndex.map( dr => {
    IndexedDataRow( dr._2, dr._1)
})



## Functions for Building Presentations



In [31]:


// http://beta.hpcc.uh.edu/scs/image/500/500/urn:cite2:hmt:vaimg.2017a:VA012RN_0013@0.04506,0.2196,0.1344,0.10093


def folioOverviewHtml( imageRois: Vector[Cite2Urn]): String = {
    val ictRoot = "http://www.homermultitext.org/ict2/index.html?urn="
    try {
        val checkForOnlyOneImage: Boolean = {
           imageRois.map(_.dropExtensions).distinct.size == 1
         }
            
        if (checkForOnlyOneImage == false) throw new Exception("More than one image represented in list.")
    
        val bigUrl = ictRoot + imageRois.mkString("&urn=")

        s"""[All Images mapped to image `${imageRois.head.dropExtensions}`.](${bigUrl})"""

    } catch {
        case e: Exception => s"Error: ${e}"
    }
}

def lineHtml( idr: IndexedDataRow): String = {
    val imgUrlBase = "http://beta.hpcc.uh.edu/scs/image/500/500/"
    val ictRoot = "http://www.homermultitext.org/ict2/index.html?urn="

    
   // s"---- \n\n ${idr.data.allenText}\n\n![${idr.data.imageRoi.get}](${imgUrlBase}${idr.data.imageRoi.get})"
    
    s"""---- \n\n `${idr.data.ctsUrn.passageComponent}`  **${idr.data.allenText}** \n\n <a href="${ictRoot}${idr.data.imageRoi.get}"><img src="${imgUrlBase}${idr.data.imageRoi.get}"/></a> """
}

def dataByEditor( data: Vector[IndexedDataRow]): Vector[Vector[IndexedDataRow]] = {
    val grouped: Vector[(String, Vector[IndexedDataRow])] = {
        data.groupBy(_.data.editor).toVector
    }
    val sortedGrouped: Vector[Vector[IndexedDataRow]] = grouped.map( g => {
        val v: Vector[IndexedDataRow] = g._2
        v.sortBy(_.index)
    })
    sortedGrouped
}

def editorPage( ed: Vector[IndexedDataRow]): String = {
    val edName: String = ed.head.data.editor
    val title: String = {
        s"## ${edName}\n\n" +
        s"*Iliad* ${ed.head.data.ctsUrn.passageComponent}" +
        "-" +
        s"${ed.last.data.ctsUrn.passageComponent}"
    }
    
    val sortedBySurface: Vector[Vector[IndexedDataRow]] = {
        ed.groupBy(_.data.surfUrn)
          .toVector
          .sortBy(_._2.head.index)
          .map( _._2 )
          .map( v => v.sortBy(_.index) )
    }
        
    
    def oneSurface( surf: Vector[IndexedDataRow] ): String = {
        
        val surfHeading = s"---\n\n---\n\n## **Folio `${surf.head.data.surfUrn}`**\n\n"
        
        val overviewLink: String = {
            val rois: Vector[Cite2Urn] = surf.map( _.data.imageRoi).filter( e => {
                e match {
                    case Some(u) => true
                    case None => false
                }
            }).map(_.get)
            folioOverviewHtml(rois)
        }
        
        val iliadLines: String = {
            surf.map( s => {
                s.data.imageRoi match {
                    case Some(u) => {
                        lineHtml(s)
                    }
                    case None => {
                        s"""--- \n\n **No line for:*** ${s.data.allenText}"""
                    }
                }
            }).mkString("\n\n")
        }
        
        surfHeading + "\n\n" + overviewLink + "\n\n" + iliadLines
        
    }
    
    val surfaceStrings: String = {
        sortedBySurface.map( sbs => {
            oneSurface(sbs)
        }).mkString("\n\n")
    }
    
    
    title + "\n\n" + surfaceStrings
}

## Assemble Reports

In [32]:
val byEditor: Vector[Vector[IndexedDataRow]] = dataByEditor(hmtData)


for ( v <- byEditor) {
    val page = editorPage(v)
    val fileIndex = f"${v.head.index}%04d"
    val fileName = fileIndex + v.head.data.editor.replaceAll("[. ,]","") + ".md"
    saveString( page, "pages/", fileName)
}